In [12]:
%load_ext autoreload
%autoreload 2
from hmpai.pytorch.utilities import *
from hmpai.pytorch.models import *
from hmpai.training import split_data_on_participants
from hmpai.normalization import *
from hmpai.visualization import add_attribution
from pathlib import Path
import xarray as xr
from hmpai.data import preprocess
from captum.attr import IntegratedGradients

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
set_global_seed(42)
data_path = Path("../data/sat1/split_stage_data.nc")
dataset = xr.load_dataset(data_path)
train_data, val_data, test_data = split_data_on_participants(dataset, 60, norm_dummy)
chk_path = Path("../models/gru/checkpoint.pt")

In [14]:
checkpoint = load_model(chk_path)

In [19]:
model_kwargs = {
    "n_channels": len(dataset.channels),
    "n_samples": len(dataset.samples),
    "n_classes": len(dataset.labels),
}
model = SAT1GRU(**model_kwargs)
model.load_state_dict(checkpoint["model_state_dict"])
model.to(DEVICE).eval()

SAT1GRU(
  (relu): ReLU()
  (gru): GRU(30, 256, batch_first=True)
  (linear): Linear(in_features=256, out_features=128, bias=True)
  (linear_final): Linear(in_features=128, out_features=5, bias=True)
)

In [20]:
ig = IntegratedGradients(model.forward)

In [22]:
attributions = add_attribution(test_data, ig)
# Fix data not on same device?

Batch: 1/125


RuntimeError: cudnn RNN backward can only be called in training mode

In [ ]:
attributions, _ = ig.attribute(
    test_data,
)